# Lab 3: Build a massive Playlist from Spotify with Artist, Title and  Audio feature

## Bringing in the Wrapper:

In [1]:
!pip install spotipy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
#secrets_file = open("secrets.txt","r")

In [3]:
secrets_file = open("/Users/Nadine Kleine/IH-Labs/secrets.txt", "r")

In [4]:
string = secrets_file.read()

In [5]:
string

'cid:85a00ad18bbc4abe8b3bdc07604afda1\ncsecret:879fe93205324ab1941d97a2fd98ff8d'

In [6]:
string.split('\n')

['cid:85a00ad18bbc4abe8b3bdc07604afda1',
 'csecret:879fe93205324ab1941d97a2fd98ff8d']

In [7]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

['cid', '85a00ad18bbc4abe8b3bdc07604afda1']
['csecret', '879fe93205324ab1941d97a2fd98ff8d']


In [8]:
secrets_dict

{'cid': '85a00ad18bbc4abe8b3bdc07604afda1',
 'csecret': '879fe93205324ab1941d97a2fd98ff8d'}

In [9]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

## Searching for playlists:

### First playlist: biggest playlist ever

In [10]:
playlist1 = sp.user_playlist_tracks("spotify", "4rnleEAOdmFAbRcNCgZMpY")

In [11]:
playlist1["total"] 

5295

In [12]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']  
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']  
        sleep(randint(1,3)) 
    return tracks

In [13]:
all_tracks = get_playlist_tracks("4rnleEAOdmFAbRcNCgZMpY")
len(all_tracks)

5295

In [14]:
title = []
artist = []
uri = []
for i in range(len(all_tracks)):
    artist.append(all_tracks[i]["track"]["album"]['artists'][0]['name'])
    title.append(all_tracks[i]['track']['name'])
    uri.append(all_tracks[i]['track']['uri'])

In [15]:
len(uri)

5295

In [16]:
import pandas as pd

In [17]:
pl1 = pd.DataFrame({'artist' : artist, 'title' : title, 'uri': uri})

In [18]:
pl1.head()

,artist,title,uri
0,Hozier,Take Me To Church,spotify:track:7dS5EaCoMnN7DzlpT6aRn2
1,Mike Posner,Cooler Than Me - Single Mix,spotify:track:2V4bv1fNWfTcyRJKmej6Sj
2,"Tyler, The Creator",See You Again (feat. Kali Uchis),spotify:track:7KA4W4McWYRpgf0fWsJZWB
3,Bastille,Pompeii,spotify:track:3gbBpTdY8lnQwqxNCcf795
4,Shakira,Hips Don't Lie (feat. Wyclef Jean),spotify:track:3ZFTkvIE7kyPt6Nu3PEa7V


In [19]:
pl1.shape

(5295, 3)

### Second playlist: 

In [20]:
playlist2 = sp.user_playlist_tracks("spotify", "1G8IpkZKobrIlXcVPoSIuf")

In [21]:
playlist2["total"] 

10000

In [22]:
all_tracks2 = get_playlist_tracks("1G8IpkZKobrIlXcVPoSIuf")
len(all_tracks2)

10000

In [23]:
title = []
artist = []
uri = []
for i in range(len(all_tracks2)):
    try:
        artist.append(all_tracks2[i]["track"]["album"]['artists'][0]['name'])
        title.append(all_tracks2[i]['track']['name'])
        uri.append(all_tracks2[i]['track']['uri'])
    except:
        artist.append('None')
        title.append('None')
        uri.append('None')

In [24]:
len(title)
len(uri)
len(artist)

10000

In [25]:
pl2 = pd.DataFrame({'artist' : artist, 'title' : title, 'uri': uri})

In [26]:
pl2.head()

,artist,title,uri
0,Bob Dylan,Like a Rolling Stone,spotify:track:3AhXZa8sUQht0UEdBJgpGc
1,Nirvana,Smells Like Teen Spirit,spotify:track:3oTlkzk1OtrhH8wBAduVEi
2,The Beatles,A Day In The Life - Remastered,spotify:track:3ZFBeIyP41HhnALjxWy1pR
3,The Beach Boys,Good Vibrations (Mono),spotify:track:5Qt4Cc66g24QWwGP3YYV9y
4,Chuck Berry,Johnny B Goode,spotify:track:7MH2ZclofPlTrZOkPzZKhK


### Concat the pl-dataframes: 

In [27]:
frames = [pl1, pl2]
  
playlist_all = pd.concat(frames)

In [28]:
playlist_all.head()

,artist,title,uri
0,Hozier,Take Me To Church,spotify:track:7dS5EaCoMnN7DzlpT6aRn2
1,Mike Posner,Cooler Than Me - Single Mix,spotify:track:2V4bv1fNWfTcyRJKmej6Sj
2,"Tyler, The Creator",See You Again (feat. Kali Uchis),spotify:track:7KA4W4McWYRpgf0fWsJZWB
3,Bastille,Pompeii,spotify:track:3gbBpTdY8lnQwqxNCcf795
4,Shakira,Hips Don't Lie (feat. Wyclef Jean),spotify:track:3ZFTkvIE7kyPt6Nu3PEa7V


In [29]:
# Check for Nans:

In [30]:
playlist_all.isna().sum().sum()

0

In [31]:
# Get rid of duplicates:

In [32]:
duplicates = playlist_all[playlist_all.duplicated('uri')]

In [33]:
duplicates

,artist,title,uri
282,Selena Gomez,Kill Em With Kindness,spotify:track:7KxhSJOYiqCDclXDBNlFSZ
404,P!nk,U + Ur Hand,spotify:track:7klL6qvAu4ZzthaUzLvtIN
561,Rihanna,SOS,spotify:track:4S5b3wwIXpVNvY2jeIQdKu
1341,Otto Knows,Next to Me,spotify:track:6ZixfeFT8V0Nle1rMZAekY
1548,SANNI,Tahdon rakastella sinua,spotify:track:6FwrG9yyKfGve1wOMq98f9
...,...,...,...
9819,None,None,None
9839,Bloodhound Gang,The Bad Touch,spotify:track:5tXeQjDShUMNSelZirUOEu
9846,None,None,None
9850,None,None,None


In [34]:
playlist_all = playlist_all.drop_duplicates(keep=False)

In [35]:
playlist_all.head()

,artist,title,uri
1,Mike Posner,Cooler Than Me - Single Mix,spotify:track:2V4bv1fNWfTcyRJKmej6Sj
2,"Tyler, The Creator",See You Again (feat. Kali Uchis),spotify:track:7KA4W4McWYRpgf0fWsJZWB
5,The Script,Hall of Fame (feat. will.i.am),spotify:track:7wMq5n8mYSKlQIGECKUgTX
6,Otto Knows,Back Where I Belong (feat. Avicii),spotify:track:78W8wiUIlQ2SnWY9TVowKZ
7,Selena Gomez,The Heart Wants What It Wants,spotify:track:2jcPoPin36hyNRSi4ZsqeB


In [60]:
playlist_all.shape

(14847, 3)

## Audio features:

In [37]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [53]:
secrets_file = open("/Users/Nadine Kleine/IH-Labs/secrets.txt", "r")

In [54]:
from random import randint
from time import sleep

In [ ]:
# first try:

features= []

for i in range(5):    
    try:
        #print(playlist_all['uri'][i])
        features.append(sp.audio_features(playlist_all['uri'][i]))
    except:
        features.append('None')

In [40]:
# second try: 
features= []

for i in range(len(playlist_all['uri'])):
    #try:
        features.append(sp.audio_features(playlist_all['uri'][i]))
    #except:
        #features.append('None')

In [55]:
# third try:
features = []
for i in playlist_all.index:
    features.append(sp.audio_features(playlist_all.at[i, 'uri']))

In [56]:
features

[[{'danceability': 0.768,
   'energy': 0.82,
   'key': 7,
   'loudness': -4.63,
   'mode': 0,
   'speechiness': 0.0474,
   'acousticness': 0.179,
   'instrumentalness': 0,
   'liveness': 0.689,
   'valence': 0.625,
   'tempo': 129.965,
   'type': 'audio_features',
   'id': '2V4bv1fNWfTcyRJKmej6Sj',
   'uri': 'spotify:track:2V4bv1fNWfTcyRJKmej6Sj',
   'track_href': 'https://api.spotify.com/v1/tracks/2V4bv1fNWfTcyRJKmej6Sj',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2V4bv1fNWfTcyRJKmej6Sj',
   'duration_ms': 213293,
   'time_signature': 4},
  {'danceability': 0.485,
   'energy': 0.863,
   'key': 1,
   'loudness': -9.027,
   'mode': 1,
   'speechiness': 0.0495,
   'acousticness': 1.22e-05,
   'instrumentalness': 0.0162,
   'liveness': 0.138,
   'valence': 0.767,
   'tempo': 116.835,
   'type': 'audio_features',
   'id': '3oTlkzk1OtrhH8wBAduVEi',
   'uri': 'spotify:track:3oTlkzk1OtrhH8wBAduVEi',
   'track_href': 'https://api.spotify.com/v1/tracks/3oTlkzk1OtrhH8wBAduVEi'

In [57]:
feature_df = pd.DataFrame(features)

In [58]:
feature_df.head()

,0,1
0,"{'danceability': 0.768, 'energy': 0.82, 'key':...","{'danceability': 0.485, 'energy': 0.863, 'key'..."
1,"{'danceability': 0.558, 'energy': 0.559, 'key'...","{'danceability': 0.364, 'energy': 0.457, 'key'..."
2,"{'danceability': 0.421, 'energy': 0.873, 'key'...","{'danceability': 0.805, 'energy': 0.558, 'key'..."
3,"{'danceability': 0.497, 'energy': 0.865, 'key'...","{'danceability': 0.511, 'energy': 0.769, 'key'..."
4,"{'danceability': 0.616, 'energy': 0.789, 'key'...","{'danceability': 0.282, 'energy': 0.716, 'key'..."


In [59]:
feature_df.shape

(14847, 2)

In [ ]:
# after we looked into the data:
# column "1" gives us information we don't need - column "0" gives us the audio feature to the uri
# So, we drop column "1":

In [62]:
feature_df.drop(feature_df.columns[1], axis=1)

,0
0,"{'danceability': 0.768, 'energy': 0.82, 'key':..."
1,"{'danceability': 0.558, 'energy': 0.559, 'key'..."
2,"{'danceability': 0.421, 'energy': 0.873, 'key'..."
3,"{'danceability': 0.497, 'energy': 0.865, 'key'..."
4,"{'danceability': 0.616, 'energy': 0.789, 'key'..."
...,...
14842,"{'danceability': 0.332, 'energy': 0.707, 'key'..."
14843,"{'danceability': 0.464, 'energy': 0.749, 'key'..."
14844,"{'danceability': 0.694, 'energy': 0.71, 'key':..."
14845,"{'danceability': 0.332, 'energy': 0.8, 'key': ..."


In [ ]:
# flatten the new dataframe:

In [63]:
from pandas.io.json import json_normalize

In [66]:
flattened_features = pd.json_normalize(feature_df[0])

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.768,0.820,7,-4.630,0,0.0474,0.179000,0.000000,0.6890,0.625,129.965,audio_features,2V4bv1fNWfTcyRJKmej6Sj,spotify:track:2V4bv1fNWfTcyRJKmej6Sj,https://api.spotify.com/v1/tracks/2V4bv1fNWfTc...,https://api.spotify.com/v1/audio-analysis/2V4b...,213293,4
1,0.558,0.559,6,-9.222,1,0.0959,0.371000,0.000007,0.1090,0.620,78.558,audio_features,7KA4W4McWYRpgf0fWsJZWB,spotify:track:7KA4W4McWYRpgf0fWsJZWB,https://api.spotify.com/v1/tracks/7KA4W4McWYRp...,https://api.spotify.com/v1/audio-analysis/7KA4...,180387,4
2,0.421,0.873,10,-4.343,1,0.0564,0.065400,0.000000,0.1230,0.629,84.786,audio_features,7wMq5n8mYSKlQIGECKUgTX,spotify:track:7wMq5n8mYSKlQIGECKUgTX,https://api.spotify.com/v1/tracks/7wMq5n8mYSKl...,https://api.spotify.com/v1/audio-analysis/7wMq...,202533,4
3,0.497,0.865,0,-3.190,1,0.0644,0.030600,0.067900,0.4430,0.595,125.960,audio_features,78W8wiUIlQ2SnWY9TVowKZ,spotify:track:78W8wiUIlQ2SnWY9TVowKZ,https://api.spotify.com/v1/tracks/78W8wiUIlQ2S...,https://api.spotify.com/v1/audio-analysis/78W8...,174203,4
4,0.616,0.789,7,-4.874,0,0.0377,0.053000,0.000000,0.1420,0.621,83.066,audio_features,2jcPoPin36hyNRSi4ZsqeB,spotify:track:2jcPoPin36hyNRSi4ZsqeB,https://api.spotify.com/v1/tracks/2jcPoPin36hy...,https://api.spotify.com/v1/audio-analysis/2jcP...,227373,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14842,0.332,0.707,7,-12.698,1,0.0361,0.000012,0.006120,0.1100,0.652,144.815,audio_features,2QSD3K3b3BJ8DPhGhQfDPW,spotify:track:2QSD3K3b3BJ8DPhGhQfDPW,https://api.spotify.com/v1/tracks/2QSD3K3b3BJ8...,https://api.spotify.com/v1/audio-analysis/2QSD...,199467,4
14843,0.464,0.749,6,-8.564,1,0.4800,0.224000,0.000046,0.3510,0.879,181.121,audio_features,49XnDVsYOHgV4gFZeCojKj,spotify:track:49XnDVsYOHgV4gFZeCojKj,https://api.spotify.com/v1/tracks/49XnDVsYOHgV...,https://api.spotify.com/v1/audio-analysis/49Xn...,201800,4
14844,0.694,0.710,2,-9.793,1,0.3400,0.527000,0.001350,0.0697,0.415,119.964,audio_features,5Y9IIH8Xmo1nuk0gfFjc4Q,spotify:track:5Y9IIH8Xmo1nuk0gfFjc4Q,https://api.spotify.com/v1/tracks/5Y9IIH8Xmo1n...,https://api.spotify.com/v1/audio-analysis/5Y9I...,183040,4
14845,0.332,0.800,1,-9.746,1,0.0326,0.000368,0.001600,0.0850,0.832,149.240,audio_features,0ya0JYEFoXNviB8RMeHDtW,spotify:track:0ya0JYEFoXNviB8RMeHDtW,https://api.spotify.com/v1/tracks/0ya0JYEFoXNv...,https://api.spotify.com/v1/audio-analysis/0ya0...,222160,4


In [86]:
flattened_features.reset_index(drop=True)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.768,0.820,7,-4.630,0,0.0474,0.179000,0.000000,0.6890,0.625,129.965,audio_features,2V4bv1fNWfTcyRJKmej6Sj,spotify:track:2V4bv1fNWfTcyRJKmej6Sj,https://api.spotify.com/v1/tracks/2V4bv1fNWfTc...,https://api.spotify.com/v1/audio-analysis/2V4b...,213293,4
1,0.558,0.559,6,-9.222,1,0.0959,0.371000,0.000007,0.1090,0.620,78.558,audio_features,7KA4W4McWYRpgf0fWsJZWB,spotify:track:7KA4W4McWYRpgf0fWsJZWB,https://api.spotify.com/v1/tracks/7KA4W4McWYRp...,https://api.spotify.com/v1/audio-analysis/7KA4...,180387,4
2,0.421,0.873,10,-4.343,1,0.0564,0.065400,0.000000,0.1230,0.629,84.786,audio_features,7wMq5n8mYSKlQIGECKUgTX,spotify:track:7wMq5n8mYSKlQIGECKUgTX,https://api.spotify.com/v1/tracks/7wMq5n8mYSKl...,https://api.spotify.com/v1/audio-analysis/7wMq...,202533,4
3,0.497,0.865,0,-3.190,1,0.0644,0.030600,0.067900,0.4430,0.595,125.960,audio_features,78W8wiUIlQ2SnWY9TVowKZ,spotify:track:78W8wiUIlQ2SnWY9TVowKZ,https://api.spotify.com/v1/tracks/78W8wiUIlQ2S...,https://api.spotify.com/v1/audio-analysis/78W8...,174203,4
4,0.616,0.789,7,-4.874,0,0.0377,0.053000,0.000000,0.1420,0.621,83.066,audio_features,2jcPoPin36hyNRSi4ZsqeB,spotify:track:2jcPoPin36hyNRSi4ZsqeB,https://api.spotify.com/v1/tracks/2jcPoPin36hy...,https://api.spotify.com/v1/audio-analysis/2jcP...,227373,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14842,0.332,0.707,7,-12.698,1,0.0361,0.000012,0.006120,0.1100,0.652,144.815,audio_features,2QSD3K3b3BJ8DPhGhQfDPW,spotify:track:2QSD3K3b3BJ8DPhGhQfDPW,https://api.spotify.com/v1/tracks/2QSD3K3b3BJ8...,https://api.spotify.com/v1/audio-analysis/2QSD...,199467,4
14843,0.464,0.749,6,-8.564,1,0.4800,0.224000,0.000046,0.3510,0.879,181.121,audio_features,49XnDVsYOHgV4gFZeCojKj,spotify:track:49XnDVsYOHgV4gFZeCojKj,https://api.spotify.com/v1/tracks/49XnDVsYOHgV...,https://api.spotify.com/v1/audio-analysis/49Xn...,201800,4
14844,0.694,0.710,2,-9.793,1,0.3400,0.527000,0.001350,0.0697,0.415,119.964,audio_features,5Y9IIH8Xmo1nuk0gfFjc4Q,spotify:track:5Y9IIH8Xmo1nuk0gfFjc4Q,https://api.spotify.com/v1/tracks/5Y9IIH8Xmo1n...,https://api.spotify.com/v1/audio-analysis/5Y9I...,183040,4
14845,0.332,0.800,1,-9.746,1,0.0326,0.000368,0.001600,0.0850,0.832,149.240,audio_features,0ya0JYEFoXNviB8RMeHDtW,spotify:track:0ya0JYEFoXNviB8RMeHDtW,https://api.spotify.com/v1/tracks/0ya0JYEFoXNv...,https://api.spotify.com/v1/audio-analysis/0ya0...,222160,4


In [85]:
playlist_all.reset_index(drop=True)

,artist,title,uri
0,Mike Posner,Cooler Than Me - Single Mix,spotify:track:2V4bv1fNWfTcyRJKmej6Sj
1,"Tyler, The Creator",See You Again (feat. Kali Uchis),spotify:track:7KA4W4McWYRpgf0fWsJZWB
2,The Script,Hall of Fame (feat. will.i.am),spotify:track:7wMq5n8mYSKlQIGECKUgTX
3,Otto Knows,Back Where I Belong (feat. Avicii),spotify:track:78W8wiUIlQ2SnWY9TVowKZ
4,Selena Gomez,The Heart Wants What It Wants,spotify:track:2jcPoPin36hyNRSi4ZsqeB
...,...,...,...
14842,Skids,Into The Valley,spotify:track:2QSD3K3b3BJ8DPhGhQfDPW
14843,Redman,Tonight's Da Night,spotify:track:49XnDVsYOHgV4gFZeCojKj
14844,FKA twigs,Figure 8,spotify:track:5Y9IIH8Xmo1nuk0gfFjc4Q
14845,The Mighty Lemon Drops,Like An Angel,spotify:track:0ya0JYEFoXNviB8RMeHDtW


In [92]:
playlist_all=playlist_all.drop('uri', axis=1)

In [ ]:
# Concat playlist_all & flattened_features

In [95]:
playlist_all=playlist_all.reset_index(drop=True)

In [96]:
playlist_all.head()

,artist,title
0,Mike Posner,Cooler Than Me - Single Mix
1,"Tyler, The Creator",See You Again (feat. Kali Uchis)
2,The Script,Hall of Fame (feat. will.i.am)
3,Otto Knows,Back Where I Belong (feat. Avicii)
4,Selena Gomez,The Heart Wants What It Wants


In [97]:
#pl_all = pd.concat([playlist_all, flattened_features], axis=1) 

frames2 = [playlist_all, flattened_features]
  
pl_all = pd.concat(frames2, axis=1)

In [98]:
pl_all.head()

,artist,title,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Mike Posner,Cooler Than Me - Single Mix,0.768,0.820,7,-4.630,0,0.0474,0.1790,0.000000,0.689,0.625,129.965,audio_features,2V4bv1fNWfTcyRJKmej6Sj,spotify:track:2V4bv1fNWfTcyRJKmej6Sj,https://api.spotify.com/v1/tracks/2V4bv1fNWfTc...,https://api.spotify.com/v1/audio-analysis/2V4b...,213293,4
1,"Tyler, The Creator",See You Again (feat. Kali Uchis),0.558,0.559,6,-9.222,1,0.0959,0.3710,0.000007,0.109,0.620,78.558,audio_features,7KA4W4McWYRpgf0fWsJZWB,spotify:track:7KA4W4McWYRpgf0fWsJZWB,https://api.spotify.com/v1/tracks/7KA4W4McWYRp...,https://api.spotify.com/v1/audio-analysis/7KA4...,180387,4
2,The Script,Hall of Fame (feat. will.i.am),0.421,0.873,10,-4.343,1,0.0564,0.0654,0.000000,0.123,0.629,84.786,audio_features,7wMq5n8mYSKlQIGECKUgTX,spotify:track:7wMq5n8mYSKlQIGECKUgTX,https://api.spotify.com/v1/tracks/7wMq5n8mYSKl...,https://api.spotify.com/v1/audio-analysis/7wMq...,202533,4
3,Otto Knows,Back Where I Belong (feat. Avicii),0.497,0.865,0,-3.190,1,0.0644,0.0306,0.067900,0.443,0.595,125.960,audio_features,78W8wiUIlQ2SnWY9TVowKZ,spotify:track:78W8wiUIlQ2SnWY9TVowKZ,https://api.spotify.com/v1/tracks/78W8wiUIlQ2S...,https://api.spotify.com/v1/audio-analysis/78W8...,174203,4
4,Selena Gomez,The Heart Wants What It Wants,0.616,0.789,7,-4.874,0,0.0377,0.0530,0.000000,0.142,0.621,83.066,audio_features,2jcPoPin36hyNRSi4ZsqeB,spotify:track:2jcPoPin36hyNRSi4ZsqeB,https://api.spotify.com/v1/tracks/2jcPoPin36hy...,https://api.spotify.com/v1/audio-analysis/2jcP...,227373,4


In [ ]:
# save flattened_features as .csv:

In [99]:
flattened_features.to_csv('audio_features.csv', index=False)

In [ ]:
# save playlist_all as .csv:

In [100]:
playlist_all.to_csv('playlist_all.csv', index=False)